In [27]:
import os
import cv2
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import RPNHead
from torchvision.models.detection.roi_heads import RoIHeads
from torchvision.models.detection.transform import GeneralizedRCNNTransform

In [28]:
# class PlayerTrackingModel(nn.Module):
#     def __init__(self, input_channels, num_classes):
#         super(PlayerTrackingModel, self).__init__()
#         # Define the layers for your player tracking model here
#         self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)  # Add max pooling layer to reduce spatial dimensions
#         self.fc1 = nn.Linear(64 * 160 * 160, 128)  # Adjust the input size based on 640x640 images
#         self.fc2 = nn.Linear(128, num_classes)  # Modify num_classes to 6

#     def forward(self, x):
#         # Define the forward pass for your model
#         x = F.relu(self.conv1(x))
#         x = self.pool(x)  # Apply max pooling to reduce spatial dimensions
#         x = F.relu(self.conv2(x))
#         x = self.pool(x)  # Apply max pooling again
#         x = x.view(x.size(0), -1)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x
# Create a Faster R-CNN model
def create_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

In [29]:
class CustomObjectDetectionDataset(Dataset):
    def __init__(self, data_dir, max_labels=6):
        self.data_dir = data_dir
        self.image_paths = []
        self.labels = []
        self.max_labels = max_labels

        for root, _, files in os.walk(os.path.join(data_dir, "images")):
            for file in files:
                if file.lower().endswith((".jpg", ".jpeg", ".png")):
                    image_path = os.path.join(root, file)
                    label_path = os.path.join(data_dir, "labels", file.replace(".jpg", ".txt"))
                    self.image_paths.append(image_path)
                    self.labels.append(label_path)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_path = self.labels[idx]

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transforms.ToTensor()(image)

        with open(label_path, 'r') as label_file:
            lines = label_file.read().splitlines()
            labels = []
            for line in lines:
                parts = line.split()
                if len(parts) == 6:  # Ensure there are 6 values in each line
                    label = [int(parts[0])] + [float(p) for p in parts[1:]]
                    labels.append(label)
            
            # Pad labels to have the same shape
            while len(labels) < self.max_labels:
                labels.append([0, 0.0, 0.0, 0.0, 0.0, 0.0])
            
            labels = labels[:self.max_labels]  # Limit to max_labels labels

        labels = torch.tensor(labels)

        return image, labels


In [30]:
# Create the PlayerTrackingModel
model = create_model(num_classes=5)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

train_data_dir = 'C:/Users/User/Desktop/UoM 3rd Year Project/Code/Football-Player-Detection-Demo-1/train'
valid_data_dir = 'C:/Users/User/Desktop/UoM 3rd Year Project/Code/Football-Player-Detection-Demo-1/valid'
test_data_dir = 'C:/Users/User/Desktop/UoM 3rd Year Project/Code/Football-Player-Detection-Demo-1/test'

In [31]:
# Define data loaders for training, validation, and test sets
train_dataset = CustomObjectDetectionDataset(train_data_dir)
valid_dataset = CustomObjectDetectionDataset(valid_data_dir)
test_dataset = CustomObjectDetectionDataset(test_data_dir)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [32]:
# Define the device (use GPU if available, otherwise use CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move your model to the specified device
model.to(device)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()

        # Ensure the labels are also moved to the same device as the model (e.g., GPU)
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images, labels)  # Provide labels to the model during training
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Training Epoch [{epoch+1}/{num_epochs}]: Loss: {average_loss:.4f}')

# Validation loop
model.eval()
total_loss = 0

with torch.no_grad():
    for images, labels in valid_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()

average_loss = total_loss / len(valid_loader)
print(f'Validation Epoch [{epoch+1}/{num_epochs}]: Loss: {average_loss:.4f}')

IndexError: too many indices for tensor of dimension 2

In [ ]:
# Testing loop
model.eval()
total_loss = 0
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

average_loss = total_loss / len(test_loader)
accuracy = (total_correct / total_samples) * 100
print(f'Testing: Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'player_tracking_model.pth')